In [1]:
import pandas as pd
import numpy as np
from src.variable_spec import VariableSpec
from itertools import product
from src.variable_transformer import transform_value

# Create a date range for 3 years of monthly data
date_range = pd.date_range(start='2020-01-01', periods=36, freq='MS')

# Create different patterns of data
np.random.seed(42)  # For reproducibility

# Linear pattern: simple increasing values
linear = np.arange(1, 37)

# Create the DataFrame
test_df = pd.DataFrame({
    'value': linear
}, index=date_range)

# Function to create all possible VariableSpec combinations
def create_all_specs(column_name='value'):
    """Create all 8 possible combinations of transformation specifications."""
    lag_orders = [0, 1]
    diff_orders = [0, 1]
    log_transforms = [False, True]
    
    all_specs = []
    for diff, log, lag in product(diff_orders, log_transforms, lag_orders):
        spec = VariableSpec(
            name=column_name,
            diff_order=diff,
            log_transform=log,
            lag_order=lag
        )
        all_specs.append(spec)
    
    return all_specs

# Create all specifications
all_specs = create_all_specs('value')

# Apply transformations and add as new columns
transformed_df = test_df.copy()

for spec in all_specs:
    
    # Create column name based on transformations
    column_name = spec.get_transformed_column_name()
    
    # Create an empty series for the transformed values
    transformed_values = pd.Series(index=test_df.index, dtype=float)
    
    # Apply transformation for each date
    for date in test_df.index:
        transformed_values[date] = transform_value(
            variable_spec=spec,
            date=date,
            base_df=test_df
        )
    
    # Add to dataframe
    transformed_df[column_name] = transformed_values

# Display the result
print(transformed_df)